In [1]:
from pathlib import Path
from encode import sample_files, get_units
import pandas as pd

audio_dir = Path("data/dev-clean")
audio_ext = ".flac"
align_path = Path("data/alignments/dev-clean/alignments.csv")
save_dir = Path("features/")
wav_dir = Path("data/dev-clean")

align_df = pd.read_csv(align_path)

In [2]:
feat_dir = Path("features/0.2")
feat_paths = list(feat_dir.rglob("**/*.npy"))
print(len(feat_paths))

63137


In [5]:
paths, sample_size = sample_files(
    audio_dir=audio_dir, audio_ext=audio_ext, sample_size=1000
)

gamma = 0.2

In [ ]:
get_units(
    paths=paths,
    align_df=align_df,
    wav_dir=wav_dir,
    gamma=gamma,
    layer=7,
    save_dir=save_dir,
)

In [4]:
from distance import process_chunks_sparse

gamma = 0.2
feat_dir = Path(f"features/{gamma}")
csv_path = Path(f"output/{gamma}/info.csv")
dist_mat_out_path = Path(f"output/{gamma}/dist_mat.npz")
chunk_limit = 1000000

process_chunks_sparse(
    feat_dir=feat_dir,
    info_csv_path=csv_path,
    dist_mat_out_path=dist_mat_out_path,
    chunk_limit=chunk_limit,
)

KeyboardInterrupt: 

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path


csv_path = "output/0.2/info.csv"
dist_mat_out_path = Path("output/0.2/dist_mat.npz")

info_df = pd.read_csv(csv_path)

In [3]:
dist_mat = np.load(dist_mat_out_path)
dist_mat = dist_mat[dist_mat.files[0]]

KeyboardInterrupt: 

In [4]:
from cluster import to_sparse_upper_chunked

sparse_dist_mat = to_sparse_upper_chunked(
    dist_mat, chunk_size=500, save_path="sparse_dist_mat.npz"
)


Converting to Sparse: 100%|██████████| 127/127 [00:01<00:00, 70.75chunk/s]


In [3]:
import scipy.sparse as sp

sparse_dist_mat = sp.load_npz("output/0.2/sparse_dist_mat.npz")

In [ ]:
def get_sim_mat(dist_mat):
    dist_mat = dist_mat.tocsr()

    # Convert distances to similarities for nonzero elements
    max_dist = dist_mat.data.max()
    similarity_matrix = dist_mat.copy()
    similarity_matrix.data = max_dist - similarity_matrix.data

    return similarity_matrix


sim_mat = get_sim_mat(sparse_dist_mat)

In [ ]:
import numpy as np

# Extract nonzero values
nonzero_values = sparse_dist_mat.data

# Compute summary statistics
mean_sim = np.mean(nonzero_values)
median_sim = np.median(nonzero_values)
min_sim = np.min(nonzero_values)
max_sim = np.max(nonzero_values)
std_sim = np.std(nonzero_values)

print(
    f"Mean: {mean_sim:.4f}, Median: {median_sim:.4f}, Min: {min_sim:.4f}, Max: {max_sim:.4f}, Std: {std_sim:.4f}"
)


Mean: 0.0077, Median: 0.0000, Min: 0.0000, Max: 1.0000, Std: 0.0860


In [ ]:
import scipy.sparse as sp
import networkx as nx


def basic_graph_clustering(similarity_matrix, threshold=0.5):
    """
    Performs basic clustering using connected components on a sparse similarity matrix.

    Parameters:
    - similarity_matrix (scipy.sparse matrix): Sparse similarity matrix.
    - threshold (float): Minimum similarity value to consider an edge.

    Returns:
    - dict: Node-to-cluster mapping.
    """
    # Ensure matrix is in CSR format
    similarity_matrix = sp.csr_matrix(similarity_matrix)

    # Create a graph where edges exist for similarity >= threshold
    sources, targets = similarity_matrix.nonzero()
    weights = similarity_matrix.data

    # Filter edges based on threshold
    edges = [(s, t) for s, t, w in zip(sources, targets, weights) if w >= threshold]

    # Build graph
    G = nx.Graph()
    G.add_edges_from(edges)

    # Find connected components (each component is a cluster)
    components = list(nx.connected_components(G))

    # Assign cluster labels
    node_to_cluster = {
        node: i for i, cluster in enumerate(components) for node in cluster
    }

    return node_to_cluster  # Dictionary {node: cluster}


# Example usage
# clusters = basic_graph_clustering(sim_matrix, threshold=0.5)


In [6]:
labels = cluster_graph(sim_mat, 13967)

Processing Similarity Matrix: 100%|██████████| 63136/63136 [00:00<00:00, 344482.00it/s]


: 

In [8]:
k = 13967
labels = basic_graph_clustering(sim_mat, 0.9)

In [9]:
print(len(labels))

62788
